# Make an Antarctic Rift Catalog

## Step 1. Read the ATL06 files into a Python data structure
Put the needed info in a dictionary, save the whole thing to file.

In [4]:
import numpy as np
import os
import arc
import importlib
importlib.reload(arc)
import json

shelf_name='brunt-riiser-ekstrom'

atl06_file_name = './atl06_' + shelf_name + '-atl06.pkl'
atl06_filelist = './filelists/' + shelf_name + '-list.json'
dataset_path = '/data/fast0/'

if os.path.isfile(atl06_file_name):
    print("Data already saved, so there's no need to ingest data. \
To repeat the data ingest, it would probably be best to change the filename of the \
existing file.")
else:
    with open(atl06_filelist,'rb') as handle:
        filelist = json.load(handle)
    arc.ingest(filelist,atl06_file_name,dataset_path)

'''
ATLAS was on safehold from June 25 to July 25, 2019.  Did we download data from this time?
'''

Working on the mask...
     Loading existing ckdt.
     Mask loaded after 6.980409 s
ERROR: Key error, ATL06_20190407203152_01460310_004_01.h5
ERROR: Key error, ATL06_20190407203152_01460310_004_01.h5
ERROR: Key error, ATL06_20190407203152_01460310_004_01.h5
ERROR: Key error, ATL06_20190407203152_01460310_004_01.h5
ERROR: Key error, ATL06_20190527053933_09000312_004_01.h5
ERROR: Key error, ATL06_20190527053933_09000312_004_01.h5
ERROR: Key error, ATL06_20190527053933_09000312_004_01.h5
ERROR: Key error, ATL06_20190527053933_09000312_004_01.h5
ERROR: Key error, ATL06_20200618123724_12820712_004_01.h5
ERROR: Key error, ATL06_20200618123724_12820712_004_01.h5
ERROR: Key error, ATL06_20200810084911_07020812_004_01.h5
ERROR: Key error, ATL06_20200810084911_07020812_004_01.h5
ERROR: Key error, ATL06_20200810084911_07020812_004_01.h5
ERROR: Key error, ATL06_20200810084911_07020812_004_01.h5
ERROR: File not found,  ATL06_20210203112931_06331010_004_01.h5
Time to read the H5 files:  18.53298734

## Step 2.  Run the rift detector

In [24]:
import time
import pandas as pd
import pickle
from functools import partial
importlib.reload(arc)
tt = time.perf_counter()

# Load data (deserialize)
with open(atl06_file_name, 'rb') as handle:
    output = pickle.load(handle)
    print('ATL06 data loaded from pickle in %f s'%(time.perf_counter()-tt))


# with_tides = output.apply (lambda row: arc.run_pyTMD(row,dataset_path+'datasets/'), axis=1)
from multiprocessing import Pool
func = partial(arc.run_pyTMD, dataset_path+'datasets/')
nproc = 8
with Pool(nproc) as p:
    with_tides = p.map(func, output)


# Find the rifts
# rift_obs = arc.get_rifts(atl06_data)

# Store the rifts in a dataframe
# rift_obs=pd.DataFrame(rift_obs)

# Or you could store it in a geopanda
# import geopandas
# rift_obs = geopandas.GeoDataFrame(rift_obs,
#                              geometry=geopandas.points_from_xy(rift_obs['x-centroid'],
#                                                                rift_obs['y-centroid']))

ATL06 data loaded from pickle in 0.343149 s


NameError: name 'Pool' is not defined

## Step 3. Save the rift_obs

In [ ]:
rift_obs_output_file_name = shelf_name + '_rift_obs.pickle'
with open(rift_obs_output_file_name, 'wb') as handle:
    pickle.dump(rift_obs, handle, protocol=pickle.HIGHEST_PROTOCOL)

## Additional Steps.

From this point, useful next steps are to quality control the data (notebook qc.ipynb) and to analyze the data (analyze_rift_measurements.ipynb)